In [2]:
import os
import pandas as pd
import nltk
import re
import numpy as np
import csv
from textblob import TextBlob
import preprocessor as p
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
from nltk.chunk import ne_chunk
from nltk.corpus import stopwords
from nltk.chunk import conlltags2tree, tree2conlltags
from collections import defaultdict
from collections import Counter
from sklearn import linear_model
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()
from sklearn.pipeline import FeatureUnion
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
from sklearn.random_projection import sparse_random_matrix
from sklearn_pandas import DataFrameMapper
import swifter
import scipy
from joblib import dump, load
import string
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

Using TensorFlow backend.
/home/ubuntu/environments/my_env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ubuntu/environments/my_env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ubuntu/environments/my_env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ubuntu/environments/

True

In [3]:
emoticons_str = r"""
    (?:
        [:=;] # Eyes
        [oO\-]? # Nose (optional)
        [D\)\]\(\]/\\OpP] # Mouth
    )"""
 
regex_str = [
    emoticons_str,
    r'<[^>]+>', # HTML tags
    r'(?:@[\w_]+)', # @-mentions
    r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # hash-tags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
 
    r'(?:(?:\d+,?)+(?:\.?\d+)?)', # numbers
    r"(?:[a-z][a-z'\-_]+[a-z])", # words with - and '
    r'(?:[\w_]+)', # other words
    r'(?:\S)' # anything else
]

tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)
emoticon_re = re.compile(r'^'+emoticons_str+'$', re.VERBOSE | re.IGNORECASE)
 
def tokenize(s):
    return tokens_re.findall(s)

def preProcessingAndPOSTagging(s):
    tokens = tokenize(s)
    tokens = ["_emot_" if emoticon_re.search(token) else token for token in tokens]  
    for i in range(len(tokens)):
      try:
        if  tokens[i][0:4] == 'http':
          tokens[i] = "http"
      except: 
        continue
      try:
        if  tokens[i][0] == '#':
          tokens[i] = "_hashtag_"
      except:
        continue
      try:
        if  tokens[i][0:2].lower() == 'rt':
          tokens[i] = "_retweet_"
      except:
        continue        
      if tokens[i] == "@handle":
        tokens[i] = "_handle_"
      elif tokens[i] == ".":
        tokens[i] = "_dot_"        
      elif tokens[i] == ",":
        tokens[i] = "_comma_"   
      elif tokens[i] == ":":
        tokens[i] = "_colon_"          
      elif tokens[i] == ";":
        tokens[i] = "_semicolon_"         
      elif tokens[i] == "!":
        tokens[i] = "_exclamation_"  
      elif tokens[i] == "?":
        tokens[i] = "_question_"          
      elif tokens[i] == "&":
        tokens[i] = "_and_"
      elif tokens[i] == "@":
        tokens[i] = "_at_" 
      elif tokens[i] == "I":
        tokens[i] = "_icap_"         
      elif tokens[i] == "i":
        tokens[i] = "_ilow_"          
      elif tokens[i] == "I'm":
        tokens[i] = "_icap_ am"         
      elif tokens[i] == "i'm":
        tokens[i] = "_ilow_ am" 
      elif tokens[i] == "w":
        try:
          if tokens[i+1] == "/": 
            tokens[i] = "_with_" 
            tokens[i+1] = "" 
        except:
          continue
      elif tokens[i].isdigit():
        tokens[i] = "_digit_" 
      elif tokens[i].isalpha():
        if tokens[i].isupper():
          flag = "_upcase_"
        else:
          flag = ""
        postag = nltk.pos_tag([tokens[i]])[0][1]
        postag = "_" + postag + "_"
        lemma = lemmatizer.lemmatize(tokens[i],'v')
        if lemma == tokens[i]:
          lemma = lemmatizer.lemmatize(tokens[i],'n')
        tokens[i] = lemma.lower()
        blob = TextBlob(tokens[i])
        polarity = blob.polarity
        if polarity > 0:
          sentiment = "_positive_"
        elif polarity < 0:
          sentiment = "_negative_"
        else:
          sentiment = "_netral_"
        tokens[i] = tokens[i] + " " + flag + " " + postag + " " + sentiment + " " + "_word_"
    text = ' '.join(tokens)
    text = re.sub(r"[0-9]+", " _digit_ ", text)
    text = re.sub('\s+', ' ', text)        
    return text   


In [4]:
#HappyEmoticons
emoticons_happy = set([
    ':-)', ':)', ';)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}',
    ':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D',
    '=-3', '=3', ':-))', ":'-)", ":')", ':*', ':^*', '>:P', ':-P', ':P', 'X-P',
    'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)',
    '<3'
    ])

emoticons_sad = set([
    ':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
    ':-[', ':-<', '=\\', '=/', '>:(', ':(', '>.<', ":'-(", ":'(", ':\\', ':-c',
    ':c', ':{', '>:\\', ';('
    ])

emoticons = emoticons_happy.union(emoticons_sad)

#Emoji patterns
emoji_pattern = re.compile("["
         u"\U0001F600-\U0001F64F"  # emoticons
         u"\U0001F300-\U0001F5FF"  # symbols & pictographs
         u"\U0001F680-\U0001F6FF"  # transport & map symbols
         u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
         u"\U00002702-\U000027B0"
         u"\U000024C2-\U0001F251"
         "]+", flags=re.UNICODE)

def further_clean_tweets(tweet):
 
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(tweet)
#after tweepy preprocessing the colon symbol left remain after      #removing mentions
    tweet = re.sub(r':', '', tweet)
    tweet = re.sub(r'‚Ä¶', '', tweet)
#replace consecutive non-ASCII characters with a space
    tweet = re.sub(r'[^\x00-\x7F]+',' ', tweet)
#remove emojis from tweet
    tweet = emoji_pattern.sub(r'', tweet)
#filter using NLTK library append it to a string
    filtered_tweet = [w for w in word_tokens if not w in stop_words]
    filtered_tweet = []
#looping through conditions
    for w in word_tokens:
#check tokens against stop words , emoticons and punctuations
        if w not in stop_words and w not in emoticons and w not in string.punctuation:
            filtered_tweet.append(w)
    return ' '.join(filtered_tweet)
    #print(word_tokens)
    #print(filtered_sentence)return tweet
    
    
def getSentimentAndSubjectivity(tweet_text):
    clean_text = p.clean(tweet_text)
    post_cleaned = further_clean_tweets(clean_text)
    blob = TextBlob(post_cleaned)
    Sentiment = blob.sentiment     
    polarity = Sentiment.polarity
    subjectivity = Sentiment.subjectivity
    return "{0:.5f}".format(polarity) +","+ "{0:.5f}".format(subjectivity)

def getPolarity(sentiment):
    return sentiment.split(",")[0]

def getSubjectivity(sentiment):
    return sentiment.split(",")[1]


In [5]:
#Pre processing as required for GLOVE

FLAGS = re.MULTILINE | re.DOTALL

def hashtag(text):
    text = text.group()
    hashtag_body = text[1:]
    if hashtag_body.isupper():
        result = "<hashtag> {}".format(hashtag_body).lower()
    else:
        result = " ".join(["<hashtag>"] + [re.sub(r"([A-Z])",r" \1", hashtag_body, flags=FLAGS)])
    return result

def allcaps(text):
    text = text.group()
    return text.lower() + " <allcaps>"


def gloveTokenize(text):
    # Different regex parts for smiley faces
    eyes = r"[8:=;]"
    nose = r"['`\-]?"

    # function so code less repetitive
    def re_sub(pattern, repl):
        return re.sub(pattern, repl, text, flags=FLAGS)
    
    text = re_sub(r"RT", "retweet")
    text = re_sub(r"https?:\/\/\S+\b|www\.(\w+\.)+\S*", "<url>")

    text = re_sub(r"@\w+", "<user>")
    text = re_sub(r"{}{}[)dD]+|[)dD]+{}{}".format(eyes, nose, nose, eyes), "<smile>")
    text = re_sub(r"{}{}p+".format(eyes, nose), "<lolface>")
    text = re_sub(r"{}{}\(+|\)+{}{}".format(eyes, nose, nose, eyes), "<sadface>")
    text = re_sub(r"{}{}[\/|l*]".format(eyes, nose), "<neutralface>")
    text = re_sub(r"/"," / ")
    text = re_sub(r"<3","<heart>")
    text = re_sub(r"[-+]?[.\d]*[\d]+[:,.\d]*", "<number>")
    text = re_sub(r"#\S+", hashtag)
    text = re_sub(r"([!?.]){2,}", r"\1 <repeat>")
    text = re_sub(r"\b(\S*?)(.)\2{2,}\b", r"\1\2 <elong>")
    # i'm to let's see ----> i 'm to let 's see
    text = re_sub(r"(\w*)('.*)(\s+)", r"\1 \2 ")
    
    text = re_sub(r"(\w+)([,!.:;])", r"\1 \2 ")
    

    ## -- I just don't understand why the Ruby script adds <allcaps> to everything so I limited the selection.
    #text = re_sub(r"([^a-z0-9()<>'`\-]){2,}", allcaps)
    text = re_sub(r"([A-Z]){2,}", allcaps)

    return text.lower()

# To get the average value of glove vector
def get_w2v_general(tweet, size, vectors, aggregation='mean'):
    tweet = gloveTokenize(tweet)
    #print("Tweet after glove Tokenization: ", tweet)
    vec = np.zeros(size).reshape((1, size))
    count = 0.
    for word in tweet.split():


        try:
            vec += vectors[word].reshape((1, size))
            count += 1.
        except:
            continue
    if aggregation == 'mean':
        if count != 0:
            vec /= count
        return vec
    elif aggregation == 'sum':
        return vec
    
def getGloveValueOfOriginalTweet(tweet):
    
    #print("Tweet at getGloveValue is: ", tweet)
    
    avg_glove_value_of_tweet = get_w2v_general(tweet, 25, glove_twitter, aggregation='mean')
    #print(avg_glove_value_of_tweet)
    glove_list = np.array(avg_glove_value_of_tweet).tolist()
    glove_list = glove_list[0]    
    complete_string = " "
    sum = 0.0
    for glove in glove_list:
        
            sum = sum + glove
            glove_str= "{0:.5f}".format(glove)        

            complete_string = complete_string + " " + glove_str
            
    
    #print("Glove String is: ", complete_string)
    #print("\n\n")
    return complete_string.split()




In [6]:
path = "train_tweets.txt"
data = pd.read_csv(path,delimiter="\t", names=["id", "tweet"])
df = pd.DataFrame(data) 

In [7]:
df = df[~df.tweet.str.startswith("RT ")]

In [8]:
df

,id,tweet
0,8746,@handle Let's try and catch up live next week!
1,8746,Going to watch Grey's on the big screen - Thur...
2,8746,@handle My pleasure Patrick....hope you are well!
3,8746,@handle Hi there! Been traveling a lot and lot...
7,8746,@handle How did u get the invite Justin?
...,...,...
328190,4357,Steelbox Demonstrates Open Video Framework wit...
328191,4357,Small Businesses Rely on Sage to Help Them Rid...
328192,4357,TimeSight Systems™ Announces Next-Generation P...
328193,4357,Diebold Makes Its Leading Monitoring Solutions...


In [11]:
#df['preprocess2'] = load("indahPreProcessed_train_noRT")

In [10]:
df['preprocess2'] = df['tweet'].swifter.apply(preProcessingAndPOSTagging)

KeyboardInterrupt: 

In [ ]:
#dump(df['preprocess2'], 'indahPreProcessed_train_noRT')

In [12]:
#df['sentiment'] = load("sentimentsPreProcessing_train_noRT")

In [ ]:
df['sentiment'] = df['tweet'].swifter.apply(getSentimentAndSubjectivity)

In [55]:
 #dump(df['sentiment'], 'sentimentsPreProcessing_train_noRT')

['sentimentsPreProcessing_train_noRT']

In [9]:
#df['polarity'] = load("polarity_train_noRT")

In [13]:
df['polarity'] = df.apply(lambda row: getPolarity(row.sentiment), axis = 1)

In [57]:
#dump(df['polarity'], "polarity_train_noRT")

['polarity_train_noRT']

In [14]:
train_texts = df['preprocess2'].astype(str)
train_tweets = df['tweet'].astype(str)

In [15]:
vfeatures = TfidfVectorizer()
posfeatures = vfeatures.fit_transform(train_texts)




In [16]:
ngramvector = TfidfVectorizer(ngram_range=(2, 3), analyzer = 'char')
ngram_features = ngramvector.fit_transform(df['tweet']) 

In [17]:
ngram_features

<307062x107968 sparse matrix of type '<class 'numpy.float64'>'
	with 42496068 stored elements in Compressed Sparse Row format>

In [18]:
xfeatures = scipy.sparse.hstack([posfeatures, ngram_features])

In [19]:
xfeatures

<307062x215687 sparse matrix of type '<class 'numpy.float64'>'
	with 48677093 stored elements in COOrdinate format>

In [16]:
subnp = df['polarity'].to_numpy().astype(float)
reshaped_subnp = subnp.reshape(-1,1)
xfeatures = scipy.sparse.hstack([xfeatures, reshaped_subnp])

In [20]:
xfeatures

<307062x215687 sparse matrix of type '<class 'numpy.float64'>'
	with 48677093 stored elements in COOrdinate format>

In [21]:
ylabel = df['id'].values

In [19]:
X_train, X_test, y_train, y_test = train_test_split(xfeatures,ylabel,train_size=0.8, shuffle = True)

In [ ]:
clf = LinearSVC(random_state=0, tol=1e-5) 
clf.fit(X_train,y_train)

In [ ]:
dump(clf, "FinalSVCClassifier")

In [ ]:
predictions = clf.predict(X_test)


In [ ]:
accuracies = accuracy_score(y_test,predictions)
print("Accuracy Score:",accuracies)

In [22]:
path_test = "test_tweets_unlabeled.txt"
data_test = pd.read_csv(path_test,sep = "\t", header = None, usecols = [0], quoting=csv.QUOTE_NONE)
df_test = pd.DataFrame(data_test) 

In [23]:
df_test['preprocess2'] = df_test[0].swifter.apply(preProcessingAndPOSTagging)
df_test['sentiment'] = df_test[0].swifter.apply(getSentimentAndSubjectivity)
df_test['polarity'] = df_test.apply(lambda row: getPolarity(row.sentiment), axis = 1)

In [35]:
test_texts = df_test['preprocess2'].astype(str)
posfeatures_test = vfeatures.transform(test_texts)

In [36]:
test_tweets = df_test[0].astype(str)
ngram_features_test = ngramvector.transform(test_tweets)

In [37]:
Xtest_tfidf_test = scipy.sparse.hstack([posfeatures_test, ngram_features_test])

subnp_test = df_test['polarity'].to_numpy().astype(float)
reshaped_subnp_test = subnp_test.reshape(-1,1)
Xtest_tfidf_test = scipy.sparse.hstack([Xtest_tfidf_test, reshaped_subnp_test])

In [38]:
predictions_test = clf.predict(Xtest_tfidf_test)

In [39]:
df_result = pd.DataFrame(predictions_test)
df_result.index = df_result.index + 1
df_result = df_result.reset_index()
df_result.rename(columns={'index':'Id',0:'Predicted'},inplace=True)
df_result.to_csv("benchMark_removingRT_2-3charLevel_SCV_100.csv", index = None, header=True)